# Overview of the GAIA dataset

In [10]:
#jsonl or JSON lines is a text file where every row is an independent JSON object

import json
# Load the metadata.jsonl file
with open('../data/metadata.jsonl', 'r') as jsonl_file:
    json_list = list(jsonl_file)


json_list[0:3]

['{"task_id": "c61d22de-5f6c-4958-a7f6-5e9707bd3466", "Question": "A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?", "Level": 2, "Final answer": "egalitarian", "file_name": "", "Annotator Metadata": {"Steps": "1. Go to arxiv.org and navigate to the Advanced Search page.\\n2. Enter \\"AI regulation\\" in the search box and select \\"All fields\\" from the dropdown.\\n3. Enter 2022-06-01 and 2022-07-01 into the date inputs, select \\"Submission date (original)\\", and submit the search.\\n4. Go through the search results to find the article that has a figure with three axes and labels on each end of the axes, titled \\"Fairness in Agreement With European Values: An Interdisciplinary Perspective on AI Regulation\\".\\n5. Note the six words use

json_list has this format

[
    
    '{"id": 1, "name": "Alice"}',

    '{"id": 2, "name": "Bob"}'
]

I want to convert it into json_QA

[

    {"id": 1, "name": "Alice"},

    {"id": 2, "name": "Bob"}
]

In [13]:
# Convert the list of strings in JSON format into a list of dictionaries 
 
json_QA = []
for json_str in json_list:
    json_data = json.loads(json_str)
    json_QA.append(json_data)

json_QA[0]

{'task_id': 'c61d22de-5f6c-4958-a7f6-5e9707bd3466',
 'Question': 'A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?',
 'Level': 2,
 'Final answer': 'egalitarian',
 'file_name': '',
 'Annotator Metadata': {'Steps': '1. Go to arxiv.org and navigate to the Advanced Search page.\n2. Enter "AI regulation" in the search box and select "All fields" from the dropdown.\n3. Enter 2022-06-01 and 2022-07-01 into the date inputs, select "Submission date (original)", and submit the search.\n4. Go through the search results to find the article that has a figure with three axes and labels on each end of the axes, titled "Fairness in Agreement With European Values: An Interdisciplinary Perspective on AI Regulation".\n5. Note the six words used as labels: deon

In [14]:
# Check all the information inside every dictionary

import random

random_samples = random.sample(json_QA, 1)
for sample in random_samples:
    print("=" * 50)
    print(f"Task ID: {sample['task_id']}")
    print(f"Question: {sample['Question']}")
    print(f"Level: {sample['Level']}")
    print(f"Final Answer: {sample['Final answer']}")
    print(f"Annotator Metadata: ")
    print(f"  ├── Steps: ")
    for step in sample['Annotator Metadata']['Steps'].split('\n'):
        print(f"  │      ├── {step}")
    print(f"  ├── Number of steps: {sample['Annotator Metadata']['Number of steps']}")
    print(f"  ├── How long did this take?: {sample['Annotator Metadata']['How long did this take?']}")
    print(f"  ├── Tools:")
    for tool in sample['Annotator Metadata']['Tools'].split('\n'):
        print(f"  │      ├── {tool}")
    print(f"  └── Number of tools: {sample['Annotator Metadata']['Number of tools']}")
print("=" * 50)

Task ID: 08f3a05f-5947-4089-a4c4-d4bcfaa6b7a0
Question: Given $x_0 = -5$ and $f(x) = x^3 + 4x^2 - 3x + 8$, what is the smallest $n$ where using Newton's Method $n = n+1$ after rounding to four decimal places?
Level: 2
Final Answer: 2
Annotator Metadata: 
  ├── Steps: 
  │      ├── 1. Verify Netwon's method as x_(n+1) = x_n - f(x_n)/f'(x_n) by searching
  │      ├── 2. Calculate the derivative: f'(x) = 3x^2 + 8x - 3
  │      ├── 3. Find x_1 using the given x_0 value: x_1 = -5 - ((-5)^3 + 4(-5)^2 - 3(-5) + 8)/(3(-5)^2 + 8(-5) - 3) = -79/16 ≈ -4.9375
  │      ├── 4. Iterate: x_2 = -79/16 - ((-79/16)^3 + 4(-79/16)^2 - 3(-79/16) + 8)/(3(-79/16)^2 + 8(-79/16) - 3) = -309711/62744 ≈ -4.9361
  │      ├── 5. They are not the same, so iterate: x_3 = -309711/62744 - ((-309711/62744)^3 + 4(-309711/62744)^2 - 3(-309711/62744) + 8)/(3(-309711/62744)^2 + 8(-309711/62744) - 3) = -18658881319456319/3780082116675876 ≈ -4.9361
  │      ├── 6. They are the same, so we stop and know n = 2 is the smallest v

From this information we need the uuid and the Question. From the question I will create the embeddings to store them into our supabase vector database.

In [4]:
#TODO finish the optional part

import pandas as pd

df = pd.read_csv("../data/supabase_docs.csv")
df

,content,metadata,embedding
0,Question : A paper about AI regulation that wa...,{'source': 'c61d22de-5f6c-4958-a7f6-5e9707bd34...,"[-0.0026346156373620033, 0.02306007593870163, ..."
1,Question : I’m researching species that became...,{'source': '17b5a6a3-bc87-42e8-b0fb-6ab0781ef2...,"[-0.0037757030222564936, 0.03832027688622475, ..."
2,Question : If we assume all articles published...,{'source': '04a04a9b-226c-43fd-b319-d5e8974367...,"[0.020039480179548264, 0.023754185065627098, -..."
3,"Question : In Unlambda, what exact charcter or...",{'source': '14569e28-c88c-43e4-8c32-097d35b9a6...,"[0.07064956426620483, -0.02905994839966297, 0...."
4,Question : If Eliud Kipchoge could maintain hi...,{'source': 'e1fc63a2-da7a-432f-be78-7c4a955987...,"[0.022228224202990532, -0.012232569046318531, ..."
...,...,...,...
160,Question : In NASA's Astronomy Picture of the ...,{'source': '0bdb7c40-671d-4ad1-9ce3-986b159c0d...,"[0.051419615745544434, -0.014581809751689434, ..."
161,"Question : In the film Goldfinger, what color ...",{'source': '08c0b6e9-1b43-4c2e-ae55-4e3fce2c27...,"[0.05897268280386925, -0.026528865098953247, 0..."
162,"Question : As of May 2023, how many stops are ...",{'source': 'db4fd70a-2d37-40ea-873f-9433dc5e30...,"[-0.02830876223742962, 0.030977396294474602, -..."
163,Question : In the 2015 Metropolitan Museum of ...,{'source': '853c8244-429e-46ca-89f2-addf40dfb2...,"[0.04488062486052513, 0.05813043937087059, -0...."
